In [1]:
import numpy as np
import pandas as pd
import scrapy
import requests
import time
import numpy as np
import xlsxwriter
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common import TimeoutException

In [4]:
base="https://eurofarma.com.br"
url_eurofarma = base+"/produtos/buscar"

In [3]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser.get(url_eurofarma)


# Aguarde um tempo suficiente para o conteúdo ser carregado
time.sleep(5)

# Defina uma lista vazia para armazenar as URLs
lista_url = []
i=0
while True:
    print(i)
    i+=1
    html_anterior = browser.page_source
    soup = BeautifulSoup(html_anterior, 'html.parser')
    for a in soup.find_all('a', href=True, class_='product-card col-xl-3 col-md-6'):
        if a['href'] != '':
            lista_url.append(a['href'])
    try:
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "btn-primary")))
    except TimeoutException:
        print("Timeout: O elemento de 'Carregar Mais' não está presente.")
    element = browser.find_element(By.CLASS_NAME, "btn-primary")
    if element.is_enabled():
        browser.execute_script("arguments[0].click();", element)
    else:
        print("ERRO Enable Click")
        break
    try:
        WebDriverWait(browser, 10).until(EC.staleness_of(element))
    except TimeoutException:
        print("Fim....")
        break
    time.sleep(2)
    html_atual = browser.page_source
    if html_anterior == html_atual:
        break
browser.quit()

[WDM] - Downloading: 100%|██████████| 6.98M/6.98M [00:07<00:00, 964kB/s] 


0
1
2
3
4
Fim....


In [5]:

CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def get_medicamento_ref(url_medicamento):

    soup_ = BeautifulSoup(requests.get(url_medicamento+"?redirectLogin=no").text, 'html.parser')
    contra_indicacao=make_info(soup_,'div','contraindication')
    indicacao=make_info(soup_,'div','indication')
    if len(contra_indicacao)>0 and len(indicacao)>0:
        classes=', '.join(list(np.array(make_info(soup_,'span','label'))[1:]))
        ativo = filter_h5(soup_)
        contra_indicacao= ''.join(contra_indicacao).replace('Contra-indicação','')
        result={'indicacao': ''.join(indicacao),'contra_indicacao':''.join(contra_indicacao),"ativo":ativo,'classes':classes}
        return result
    else:
        return None

def filter_h5(soup_):
    h5_element=soup_.find('h5', string='Princípio Ativo')
    if(h5_element):
        span_element = h5_element.find_next_sibling('span').string
        return  span_element
    return ""

def make_info(soup_,tag,class_):
    result=soup_.find_all(tag,  class_=class_)
    contents = [div.get_text(strip=True) for div in result]
    contents_filter = [re.sub(CLEANR, '', content) for content in contents]
    contents_filter = [content.replace('\r\n', ' ') for content in contents_filter]
    return contents_filter

test=lista_url[1]
print(test)
get_medicamento_ref(base+test)

/produtos/aceclofenaco-100-mg


{'indicacao': 'O aceclofenaco está indicado para o tratamento de processos dolorosos e inflamatórios tais como: dores de dentes, traumatismos, dores musculares (ex: lombares), dores pós-cirúrgicas (após o parto normal, após extração dentária), dores nas articulações dos ombros e reumatismos. Também é eficaz no tratamento crônico de processos inflamatórios, como artrite reumatoide, osteoartrite e espondilite anquilosante.',
 'contra_indicacao': 'Este medicamento é contraindicado em caso de hipersensibilidade (alergia) conhecida ao aceclofenaco e/ou a qualquer componente da formulação. O aceclofenaco não deve ser administrado em pacientes alérgicos ao diclofenaco, pois se relata a ocorrência de reações alérgicas graves nestes pacientes, quando em tratamento com agentes anti-inflamatórios não esteroidais. Também não deve ser administrado àqueles que sofrem de broncoespasmo, urticária ou rinite aguda devido ao ácido acetilsalicílico ou a outros anti-inflamatórios não-esteroidais. Este medi

In [31]:
list_medicamento_resumo = pd.DataFrame(columns=["nome","ativo","indicacao","contra_indicacao","classes"])

In [32]:
with open("./lista_url", "r") as file:
    lista_url = file.readlines()
    lista_url = [line.strip() for line in lista_url]
# lista_url=np.array(open("./lista_url",'r',encoding="utf-8").readlines())
print(lista_url[0])
i=0
for url_ in list(np.array(lista_url)):
    remedio=get_medicamento_ref(base+url_)
    nome=str(url_).split("/")[len(str(url_).split("/"))-1]
    if remedio:
        print(f"index {i} medicamento: {nome}")
        indicacao=remedio['indicacao']
        contra_indicacao=remedio['contra_indicacao']
        ativo=remedio['ativo']
        classes =remedio['classes']
        list_medicamento_resumo = list_medicamento_resumo._append({'nome':nome,"ativo":ativo,"indicacao":indicacao,"contra_indicacao":contra_indicacao,"classes":classes},ignore_index=True)
    else:
        print(f"index {i} medicamento: {nome} sem informações na pagina")
    i+=1

/produtos/trimusk
index 0 medicamento: trimusk
index 1 medicamento: floripa-1
index 2 medicamento: versa
index 3 medicamento: bicerto
index 4 medicamento: fosfato-dissdico-de-dexametasona-cloridrato-de-tiamina-cloridrato-de-piridoxina-cianocobalamina
index 5 medicamento: molieri-30
index 6 medicamento: ceftriaxona-sodica-i-v
index 7 medicamento: alprazolam
index 8 medicamento: berk
index 9 medicamento: calnate
index 10 medicamento: valerato-de-betametasona-sulfato-de-gentamicina-tolnaftato-clioquinol-2
index 11 medicamento: letrozol
index 12 medicamento: tinidazol-nitrato-de-miconazol
index 13 medicamento: promestrieno
index 14 medicamento: losartana-potassica
index 15 medicamento: pondera-10mg-30mg
index 16 medicamento: snif-jet-9mg
index 17 medicamento: cloridrato-de-sibutramina
index 18 medicamento: mesilato-de-imatinibe
index 19 medicamento: duloxetina-cloridrato-de
index 20 medicamento: linezolida
index 21 medicamento: desloratadina-xpe
index 22 medicamento: montelucaste-de-sodio-

In [34]:
list_medicamento_resumo.to_excel("medicamentos.xlsx",index=False,engine='xlsxwriter')